主要是讲 theo 的模型和 lx 自己编写的几个关键模块对比进行理解

In [ ]:
# 添加基本的测试操作部分
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint


LAGTPKS = np.random.rand(1, 7)

time = np.linspace(0, 100, 1000)

    # for i in range(num):
    #     x0 = LAGTPKS[i]
    #     traj = odeint(self.dDynVar, x0, time)

    #     # 遍历绘制 LAGTPKS 的每个变量数值随时间变化的曲线
    #     for j in range(7):
    #         ax.plot(time, traj[:, j])

x0 = LAGTPKS[0]
traj = odeint(self.dDynVar, x0, time)



def dynamic_eqs(self, LAGTPKS, t) -> list:
    """The differential equations we have to solve for the next step"""

    # We define variables for simpler notation
    def phot(A, T):
        """Photosynthesis"""
        return (self.params['l0'] - self.params['lT'] * T) * np.sqrt(A / self.params['Sigma'])

    def resp(T):
        """Respiration"""
        return self.params['a0'] + self.params['aT'] * T

    def diff(L, A, G=0.):
        """Diffusion between atmosphere and ocean"""
        return self.params['delta'] * (self.params['Cstar'] - L - G - (1 + self.params['m']) * A)

    def fert(W):
        """Human fertility rate"""
        return 2 * self.params['p'] * self.params['Wp'] * W / (self.params['Wp']**2 + W**2)

    def mort(W):
        """Human mortality rate"""
        return self.params['q0'] / W + self.params['wL']/self.params['Sigma']

    # We unpack the state variables and derived variables
    L, A, G, T, P, K, S = LAGTPKS
    
    B, F, R, Y, W = self.compute_derived_variables()

    # clamp here
    L = np.amin([np.amax([L, 1e-12]), self.params['Cstar']])
    A = np.amin([np.amax([A, 1e-12]), self.params['Cstar']])
    G = np.amin([np.amax([G, 1e-12]), self.params['Cstar']])
    T = np.amax([T, 1e-12])
    P = np.amax([P, 1e-12])
    K = np.amax([K, 1e-12])
    S = np.amax([S, 1e-12])

    # odes to solve
    dL = (phot(A, T) - resp(T)) * L - B
    dA = -dL + diff(L, A, G=G)
    dG = -F
    dT = self.params['g'] * (A / self.params['Sigma'] - T)
    dP = P * (fert(W) - mort(W))
    dK = self.params['i'] * Y - self.params['k0'] * K
    dS = self.params['sR'] * R - self.params['sS'] * S

    return [dL, dA, dG, dT, dP, dK, dS] # 关键的 ode 计算的部分